In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1GhyH4k9C4uPRnMAMKhJYOqa-V9Tqt4q8' ### File ID ###
data = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))


Designing NN

In [3]:
import tensorflow as tf
import os
import sys, json
from pprint import pprint
import requests
import pandas as pd
import numpy as np
import torch
!pip install flair
import flair

    100% |████████████████████████████████| 71kB 4.4MB/s 
    100% |████████████████████████████████| 22.6MB 1.7MB/s 
    100% |████████████████████████████████| 51kB 10.8MB/s 
    100% |████████████████████████████████| 12.8MB 2.9MB/s 
    100% |████████████████████████████████| 798kB 24.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/54/1c/24/39ec59521bb1a83306cf5224a50f99aad60b55dd3b42d32ed0
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built flair segtok mpld3 sqlitedict
yellowbrick 0.9.1 has requirement matplotlib!=3.0.0,>=1.5.1, but you'll have matplotlib 3.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
spacy

In [4]:
n_input, n_hidden, n_output = 5,3,1

### initialize tensor parameters
x = torch.randn((1, n_input))
y = torch.randn((1, n_output))
np.shape(x)

torch.Size([1, 5])

In [0]:
## initialize weights and biases
w1 = torch.randn((n_input, n_hidden))
w2 = torch.randn((n_hidden, n_output))
b1 = torch.randn((1, n_hidden))
b2 = torch.randn((1, n_output))

In [6]:
### Sigmoid activation function
def sigmoid_activation(z):
  return 1/(1 + torch.exp(-z))

### Activation of hidden layer
z1 = torch.mm(x, w1) + b1
a1 = sigmoid_activation(z1)

### Activation of output layer
z2 = torch.mm(a1, w2) + b2
output = sigmoid_activation(z2)

z1, z2, a1, output

(tensor([[ 3.1421,  6.8952, -2.7946]]),
 tensor([[0.1612]]),
 tensor([[0.9586, 0.9990, 0.0576]]),
 tensor([[0.5402]]))

In [0]:
### Loss calculation
loss = y-output

In [0]:
### Backpropogation
### Derivative of activation
def sigmoid_delta(x):
  return x*(1-x)

In [0]:
### Derivative of error terms
delta_output = sigmoid_delta(output)
delta_hidden = sigmoid_delta(a1)

In [0]:
### Backpropagate erors to previous layer
d_output = loss * delta_output
loss_h = torch.mm(d_output, w2.t())
d_hid = loss_h * delta_hidden

In [0]:
learning_rate = 0.1

In [12]:
### Updating weights and biases
w2 += torch.mm(a1.t(), d_output)*learning_rate
w1 += torch.mm(x.t(), d_hid)*learning_rate

b2 += d_output.sum()*learning_rate
b1 += d_hid.sum()*learning_rate
w1, w2, b1, b2

(tensor([[-0.9253, -0.7148,  0.5233],
         [-1.0332, -0.8747, -0.7725],
         [-1.9024, -0.5898,  0.4298],
         [ 2.8668, -0.5807,  0.0718],
         [-2.1285,  2.3066, -1.5187]]), tensor([[-1.6881],
         [ 1.2383],
         [-1.2532]]), tensor([[-0.5927,  1.0399,  1.4411]]), tensor([[0.5510]]))

In [13]:
from torchvision import transforms
from torchvision.datasets import MNIST
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
### Load and transfrom MNIST dataset
mnist = MNIST("anand", download=True, train=True, transform=_tasks)
mnist

Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: anand
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [0]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
### Paartition of data into training and validation sets
split = int(0.8*len(mnist))
index_list = list(range(len(mnist)))
train_idx, validate_idx = index_list[:split], index_list[split:]

In [0]:
### Create random sampler using SubsetRandomSampler
tr_sampler = SubsetRandomSampler(train_idx)
validate_sampler = SubsetRandomSampler(validate_idx)

In [0]:
### Create iterator objects to train and validate
train_loader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validate_loader = DataLoader(mnist, batch_size=256, sampler=validate_sampler)

In [0]:
### Use nn module to design neural network
from torch import nn
import torch.nn.functional as F
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.hidden = nn.Linear(784, 128)
    self.output = nn.Linear(128, 10)
  def forward(self, x):
      x = self.hidden(x)
      x = F.sigmoid(x)
      x = self.output(x)
      return(x)


In [19]:
model = Model()
print(model)

Model(
  (hidden): Linear(in_features=784, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)


In [0]:
### Define loss function using nn and optim
from torch import optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

In [21]:
### Train the model
for epoch in range(1,11):
  train_loss, valid_loss = [],[]
  model.train()
  
  for data, target in train_loader:
#     print(data)
#     print(np.shape(data))
    data = data.view(data.size(0), -1)
    
    optimizer.zero_grad()
    
    ### 1. Forward propagation
    output = model(data)
#     print(output)
    ### 1. Loss calculation
    loss = loss_function(output, target)
#     print(loss)
    ### 3. Backward propagation
    loss.backward()
    
    ### 4. Weight optimization
    optimizer.step()
    
    train_loss.append(loss.item())
    
  ### Evaluate model
  for data, target in validate_loader:
    data = data.view(data.size(0), -1)
    output = model(data)
    loss = loss_function(output, target)
    valid_loss.append(loss.item())
    
  print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))    

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1 Training Loss:  1.4197915561021643 Valid Loss:  0.7011252286586356
Epoch: 2 Training Loss:  0.5740270075645852 Valid Loss:  0.4462369103380974
Epoch: 3 Training Loss:  0.43209435568845017 Valid Loss:  0.37327348171396457
Epoch: 4 Training Loss:  0.37680654584410345 Valid Loss:  0.3394255562031523
Epoch: 5 Training Loss:  0.34642054037527836 Valid Loss:  0.3151104932135724
Epoch: 6 Training Loss:  0.32557326777184264 Valid Loss:  0.3000430702529055
Epoch: 7 Training Loss:  0.3098732462113208 Valid Loss:  0.28814307520998284
Epoch: 8 Training Loss:  0.2971622065660801 Valid Loss:  0.27649887793875755
Epoch: 9 Training Loss:  0.2851173348249273 Valid Loss:  0.2670294461732215
Epoch: 10 Training Loss:  0.27546708610780696 Valid Loss:  0.2586709264111012


In [26]:
dataiter = iter(validate_loader)
data, labels = dataiter.next()
data = data.view(data.size(0), -1)

output = model(data)
output

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[-0.6360, -1.9837, -0.7877,  ..., -0.9286,  0.3953, -0.1565],
        [-0.7203, -4.2173, -1.7800,  ...,  9.3168, -1.3906,  3.9490],
        [ 1.9436,  0.3520,  6.8772,  ..., -7.0824,  0.7986, -5.1118],
        ...,
        [-6.3631,  7.2642,  2.2599,  ...,  0.8058,  0.8385, -1.6629],
        [ 2.3180, -5.4311,  6.8661,  ..., -5.1892,  0.8406, -2.3909],
        [-0.0734, -5.3239, -0.5281,  ..., -2.7174,  6.6580,  1.7022]],
       grad_fn=<AddmmBackward>)

In [28]:
_,pred_tensor = torch.max(output, 1)
preds = np.squeeze(pred_tensor.numpy())


array([3, 7, 2, 5, 3, 3, 3, 4, 1, 6, 4, 1, 4, 2, 9, 5, 0, 1, 8, 7, 6, 7,
       0, 9, 1, 6, 3, 5, 6, 8, 2, 5, 6, 4, 5, 3, 2, 8, 4, 1, 7, 3, 7, 1,
       7, 1, 8, 6, 0, 1, 1, 4, 3, 6, 0, 3, 0, 3, 6, 9, 2, 9, 3, 5, 2, 6,
       2, 4, 9, 7, 2, 4, 7, 4, 3, 9, 3, 9, 0, 0, 1, 7, 5, 6, 0, 6, 8, 8,
       1, 7, 6, 7, 5, 5, 8, 7, 8, 2, 7, 5, 4, 6, 6, 2, 8, 7, 8, 7, 1, 5,
       9, 4, 7, 1, 6, 7, 7, 0, 9, 4, 0, 0, 9, 4, 1, 8, 9, 0, 5, 1, 4, 2,
       7, 5, 6, 3, 1, 9, 7, 3, 2, 3, 7, 3, 1, 9, 7, 0, 1, 5, 8, 1, 9, 0,
       9, 3, 2, 2, 8, 3, 8, 0, 7, 9, 4, 2, 6, 7, 5, 2, 7, 4, 1, 7, 7, 4,
       5, 3, 5, 1, 1, 6, 3, 0, 8, 3, 5, 6, 4, 1, 7, 8, 7, 9, 4, 3, 2, 6,
       7, 2, 5, 9, 0, 7, 7, 3, 5, 9, 7, 7, 8, 4, 6, 6, 6, 9, 2, 7, 8, 8,
       1, 4, 3, 2, 8, 8, 3, 3, 2, 7, 3, 9, 6, 8, 5, 1, 2, 1, 5, 5, 0, 7,
       5, 3, 3, 1, 1, 7, 7, 2, 0, 3, 1, 1, 2, 8])

In [34]:
print("Actual: ", labels[:10])
print("Prediction:    ", preds[:10])
labels = np.squeeze(labels.numpy())
print(labels - preds)

Actual:  tensor([3, 7, 2, 3, 3, 3, 3, 4, 1, 3])
Prediction:     [3 7 2 5 3 3 3 4 1 6]
[ 0  0  0 -2  0  0  0  0  0 -3  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  5
  0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  2 -1
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -5  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  4  0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  5  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -5  0
  0  0  0  2  6  0  0  0  0  0  0 -1  3  0  0  0]
